# Create Expert Personas to Answer Initial Questionaire

In [114]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

In [116]:
with open("outputs/1_question.md", "r") as f:
    question = f.read()

with open("outputs/2_key_themes.md", "r") as f:
    key_themes = f.read()

with open("outputs/3_questionnaire.md", "r") as f:
    questionare = f.read()

## Make an Initial Overview of Expert Types


the types of experts we might need -- give a high level overview of the experts we might need.
This is needed as ChatGPT cannot make all detailed personas at once - we need to upramp the amount of text made.

In [117]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/make_initial_overview_of_expert_types.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """can you craft descriptions of the different type of experts which would make up a diverse panel of experts on the subject of '{question}', with the key themes being '{key_themes}' where the questions being answered are '{questionare}'"""
            ),
    ]
)


llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!
expert_personas_overview_md = llm.invoke(chat_template.format_messages(
    question=question, key_themes=key_themes, questionare=questionare)).content

with open("outputs/4_expert_personas_overview.md", "w") as text_file:
    text_file.write(expert_personas_overview_md)


## Convert Initial Overview to JSON

In [118]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/convert_initial_overview_to_JSON.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert the following into a JSON object: {expert_personas_overview_md}"""
            ),
    ]
)

llm = ChatOpenAI() ## make this defined ones and used again and again!!
expert_personas_overview_json = llm.invoke(chat_template.format_messages(
    expert_personas_overview_md=expert_personas_overview_md)).content


persona_overview_json = json.loads(expert_personas_overview_json)

## Craft Expert Persona Descriptions from Overview

In [120]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/craft_persona_from_overview_persona.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert craft various LLM personas for the role of {role}, with the characteristics of {characteristics}. 
            Which could suitably answer the question of '{question}', where the key themes are '{key_themes}' and the questions being asked are '{questionare}'"""
            ),
    ]
)

llm = ChatOpenAI(model='gpt-4-turbo-preview')

In [121]:
import glob

files = glob.glob('outputs/5_custom_personas/*')
for f in files:
    os.remove(f)

for i in persona_overview_json['Expert Personas']:
    print(i['Role'])
    specific_personas = llm.invoke(chat_template.format_messages(
    role=i['Role'], 
    characteristics=i['characteristics'], 
    question=question, key_themes=key_themes, questionare=questionare)).content

    with open(f"outputs/5_custom_personas/{i['Role']}.txt", "w") as text_file:
        text_file.write(specific_personas)

Machine Learning Engineers
Data Privacy and Security Specialists
IT Infrastructure Architects
Ethics and Bias Reduction Experts
User Experience (UX) Designers
Regulatory Compliance Consultants
Business Analysts and ROI Experts
Cloud Computing and Deployment Specialists
Stakeholder Engagement Coordinators
Industry Veterans


## Convert Expert Persona Descriptions to JSON

In [124]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/convert_expert_persona_descriptions_to_JSON.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert the following into a JSON object: {specific_personas}"""
            ),
    ]
)

# with open("outputs/specific_personas.txt", "r") as text_file:
#     specific_personas = text_file.read()

llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!


files = glob.glob('outputs/5_custom_personas/*.txt')
for file in files:
    print(os.path.basename(file).replace('.txt',''))
    with open(file, 'r') as text_file:
        specific_personas = text_file.read()


    specific_personas_json = llm.invoke(chat_template.format_messages(
        specific_personas=specific_personas)).content


    specific_personas_json = json.loads(specific_personas_json)
    with open(f"outputs/5_custom_personas/{os.path.basename(file).replace('.txt','')}.json", 'w') as f:
        json.dump(specific_personas_json, f)

Regulatory Compliance Consultants
Business Analysts and ROI Experts
Cloud Computing and Deployment Specialists
Stakeholder Engagement Coordinators
Ethics and Bias Reduction Experts
Data Privacy and Security Specialists
Industry Veterans
User Experience (UX) Designers
Machine Learning Engineers
IT Infrastructure Architects


## Flesh Out Expert Personas with Viewpoints

This step adds different viewpoints, beliefs and behavioural characteristsics to personas in order to get different perspectives on the questionnaire with the same expertise.

In [131]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/flesh_out_expert_personas_with_viewpoints.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert craft a detailed personas for the role of {role}, with the characteristics of {characteristics}. 
            Which could suitably answer questions where the key themes are '{key_themes}'"""
            ),
    ]
)

llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!



In [132]:
files = glob.glob('outputs/6_final_personas/*.md')
for f in files:
    os.remove(f)

files = glob.glob('outputs/5_custom_personas/*.json')
for file in files:
    print(os.path.basename(file).replace('.json',''))
    with open(file, 'r') as j:
        specific_personas_json = json.loads(j.read())

    for i in specific_personas_json['Expert Personas']:
        print(f"  -  {i['Role']}")
        detailed_persona = llm.invoke(chat_template.format_messages(
            role=i['Role'], 
            characteristics=i['characteristics'], 
            question=question, key_themes=key_themes, questionare=questionare)).content

        with open(f"outputs/6_final_personas/{os.path.basename(file).replace('.json','')} {i['Role'].replace(':', '')}.md", "w") as text_file:
            text_file.write(detailed_persona)


Cloud Computing and Deployment Specialists
  -  Cloud Architect: The Strategic Visionary
  -  Data Privacy and Security Specialist: The Guardian
  -  Machine Learning Deployment Engineer: The Pragmatist
  -  Ethical AI Advocate: The Conscientious
  -  Stakeholder Engagement Facilitator: The Collaborator
Data Privacy and Security Specialists
  -  Data Privacy Guardian
  -  Scalability Architect
  -  AI Compliance Strategist
  -  Machine Learning Ethicist
  -  Continuous Learning Innovator
  -  Deployment Efficiency Expert
Stakeholder Engagement Coordinators
  -  The Integrator
  -  The Technophile
  -  The Guardian
Machine Learning Engineers
  -  The Data Guardian
  -  The Scalability Architect
  -  The Continuous Learner
  -  The Integration Expert
  -  The Ethical Innovator
Industry Veterans
  -  The Seasoned Strategist
  -  The Tech-Savvy Innovator
  -  The Data Protection Guardian
  -  The Process Optimization Expert
IT Infrastructure Architects
  -  The Data Guardian
  -  The Scala